# Likelihood en Model Onzekerheid

In dit labo leer je hoe je maximum likelihood schatting toepast, onzekerheid kwantificeert via betrouwbaarheidsintervallen en predictie-intervallen, en modellen vergelijkt met de likelihood ratio test.

**Leerdoelen:**
- Maximum likelihood schatting toepassen op lineaire regressie
- Standaardfouten en betrouwbaarheidsintervallen berekenen voor parameters
- Predictie-intervallen construeren voor nieuwe observaties
- Het verschil begrijpen tussen betrouwbaarheids- en predictie-intervallen
- Likelihood ratio tests gebruiken om modellen te vergelijken
- Fisher informatie interpreteren

In [ ]:
import numpy as np
from scipy import stats

## ✍️ Maximum Likelihood Schatting

Een vastgoedmakelaar verzamelt data over huizen. Je hebt de volgende data:
- **Oppervlakte** (in m²): [50, 60, 70, 80, 90, 100, 110, 120, 130, 140]
- **Prijs** (in €1000): [150, 180, 195, 220, 245, 260, 285, 300, 320, 340]
 
Bereken de Maximum Likelihood schattingen voor het lineaire model

$$
\begin{align}
\text{Prijs} &= b_0 + b_1 \times \text{Oppervlakte} + e \cr
e &\sim \mathcal{N}(0, \sigma^2)
\end{align}
$$

## ✍️ Betrouwbaarheidsintervallen

Bereken de 95% betrouwbaarheidsintervallen voor $b_0$ en $b_1$.

## ✍️ Predictie-intervallen

1. Bereken het 95% predictie-interval voor een huis van 95 m²
2. Vergelijk dit met het betrouwbaarheidsinterval voor het gemiddelde van alle huizen van 95 m²

## ✍️ Likelihood Ratio Test

We hebben twee modellen:
- **Model 0 (enkel intercept)**:

$$\text{Prijs} = b_0 + e$$

- **Model 1**:
$$\text{Prijs} = b_0 + b_1 \times \text{Oppervlakte} + e$$

Test of de oppervlakte significant bijdraagt aan het model (α = 0.05).